In [1]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import IPython.display as ipd
from tqdm import tqdm

In [2]:
from hnr import *
from jitters import *
from shimmers import *
from sound import Waveform

In [3]:
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

2023-12-05 10:25:12.051958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 10:25:12.633360: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 10:25:14.920277: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-05 10:25:14.920762: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
data_path = "/home/bmis/Documents/AI-Workspace/ALS/bmis_data/All"

control_path = 'Control'
als_with_dysarthria_path = 'ALSwDysarthria'
als_without_dysarthria_path = 'ALSwoDysarthria'

control_dir = os.path.join(data_path, control_path)
control = os.listdir(control_dir)[0]

als_with_dir = os.path.join(data_path, als_with_dysarthria_path)
als_with_dys = os.listdir(als_with_dir)[1]

als_without_dir = os.path.join(data_path, als_without_dysarthria_path)
als_without_dys = os.listdir(als_without_dir)[1]

In [5]:
def extract_td_feature(audio_path):
    td_features = []
    _, sample_rate = librosa.load(audio_path)
    #print(audio_path)
    sound = Waveform(path=audio_path, sample_rate=sample_rate)

    td_features.append(sound.jitters()['localabsoluteJitter'])
    td_features.append(sound.jitters()['localJitter'])
    td_features.append(sound.jitters()['rapJitter'])
    td_features.append(sound.jitters()['ppq5Jitter'])

    td_features.append(sound.shimmers()['localShimmer'])
    td_features.append(sound.shimmers()['localdbShimmer'])
    td_features.append(sound.shimmers()['apq3Shimmer'])
    td_features.append(sound.shimmers()['apq5Shimmer'])
    td_features.append(sound.shimmers()['apq11Shimmer'])

    _ =(sound.hnr())
    hnr = sound.hnr()
    td_features.append(hnr)

    return np.array(td_features)

In [6]:
def get_all_td_features(data_path, label):
    data = []
    labels = []

    for file in tqdm(os.listdir(data_path)):
        #print(f'loading file: {file}')
        try:
            feature = extract_td_feature(os.path.join(data_path, file))
            data.append(feature)
            labels.append(label)
        except:
            print("Error encountered while parsing file: ", file)
            continue
    return data, labels

In [7]:
def extract_MFCC_features(audio_path):
    audio, sample_rate = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=128)
    #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
    return mfccs_scaled


def get_MFCC_data(data_path, label):
    data = []
    labels = []
    for file in tqdm(os.listdir(data_path)):
        try:
            feature = extract_MFCC_features(os.path.join(data_path, file))
            data.append(feature)
            labels.append(label)
        except:
            print("Error encountered while parsing file: ", file)
            continue
    return data, labels

## 2 CLASS CLASSIFICATION USING LSTM

In [8]:
#control_data, control_labels = get_all_td_features(control_dir, 0)
#als_with_data, als_with_labels = get_all_td_features(als_with_dir, 1)
#als_without_data, als_without_labels = get_all_td_features(als_without_dir, 1)

control_data, control_labels = get_MFCC_data(control_dir, 0)
als_with_data, als_with_labels = get_MFCC_data(als_with_dir, 1)
als_without_data, als_without_labels = get_MFCC_data(als_without_dir, 1)

100%|██████████| 176/176 [00:04<00:00, 37.78it/s]


In [11]:
X = np.concatenate([control_data, als_with_data, als_without_data], axis=0)
y = np.concatenate([control_labels, als_with_labels, als_without_labels], axis=0)
X = np.array(X)

In [13]:
# Data Cleaning

"""
X[X == None] = np.nan
X = np.nan_to_num(X, nan=0.0)
df = pd.DataFrame(X)
df.fillna(0.0, inplace=True)
X = df.to_numpy()

X = X.reshape(-1,10,1)
"""
X = X.reshape(-1,128,1)
X, y = shuffle(X, y, random_state=0)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(128,1)))  # 64 units in LSTM layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Output layer with 1 neuron for binary classification
# Print model summary
model.summary()

2023-12-05 10:26:46.836209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 10:26:47.872286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-12-05 10:26:47.872317: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-05 10:26:47.873087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6

In [15]:
num_folds = 10
fold_no = 1
kfold = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []
loss_per_fold = []

# Model Hyperparameters Turning
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1,
                                                 restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('lstm_two_class_model.h5', monitor='loss', verbose=1,
                                                      save_best_only=True)

for train, test in kfold.split(X, y):
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('---------------------------------------------------')

    history = model.fit(X[train], y[train], epochs=100, verbose=1,
                        callbacks=[earlystopping, model_checkpoint])

    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    ###
    y_pred = model.predict(X[test])
    y_pred = np.round(y_pred).astype(int)
    
    precision = precision_score(y[test], y_pred)
    recall = recall_score(y[test], y_pred)
    f1 = f1_score(y[test], y_pred)

    

# Plot the confusion matrix

    #print('***************************************************')
    #print(f'Precision: {precision} -> Recall: {recall} -> F1: {f1}')
    #print('***************************************************')
    ##
    accuracy_per_fold.append(scores[1] * 100)
    precision_per_fold.append(precision)
    recall_per_fold.append(recall)
    f1_per_fold.append(f1)
    loss_per_fold.append(scores[0])

    
    
    fold_no += 1

---------------------------------------------------
Epoch 1/100
19/19 [==============================] - ETA: 0s - loss: 0.6651 - accuracy: 0.6845
Epoch 1: loss improved from inf to 0.66507, saving model to lstm_two_class_model.h5
19/19 [==============================] - 3s 97ms/step - loss: 0.6651 - accuracy: 0.6845
Epoch 2/100
19/19 [==============================] - ETA: 0s - loss: 0.6279 - accuracy: 0.7012
Epoch 2: loss improved from 0.66507 to 0.62785, saving model to lstm_two_class_model.h5
19/19 [==============================] - 2s 97ms/step - loss: 0.6279 - accuracy: 0.7012
Epoch 3/100
19/19 [==============================] - ETA: 0s - loss: 0.6348 - accuracy: 0.6995
Epoch 3: loss did not improve from 0.62785
19/19 [==============================] - 2s 100ms/step - loss: 0.6348 - accuracy: 0.6995
Epoch 4/100
19/19 [==============================] - ETA: 0s - loss: 0.6262 - accuracy: 0.6995
Epoch 4: loss improved from 0.62785 to 0.62622, saving model to lstm_two_class_model.h5


In [16]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
    print(f'> Fold {i+1} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('-----------------------------------------------')

Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.5973783731460571 - Accuracy: 71.6417908668518%
> Fold 1 - Precision: 0.7164179104477612 - Recall: 1.0 - F1: 0.8347826086956522
-----------------------------------------------
> Fold 2 - Loss: 0.5923601388931274 - Accuracy: 70.14925479888916%
> Fold 2 - Precision: 0.7014925373134329 - Recall: 1.0 - F1: 0.8245614035087719
-----------------------------------------------
> Fold 3 - Loss: 0.5480514764785767 - Accuracy: 77.61194109916687%
> Fold 3 - Precision: 0.7924528301886793 - Recall: 0.9130434782608695 - F1: 0.8484848484848485
-----------------------------------------------
> Fold 4 - Loss: 0.5564599633216858 - Accuracy: 76.11940503120422%
> Fold 4 - Precision: 0.8367346938775511 - Recall: 0.8367346938775511 - F1: 0.8367346938775511
-----------------------------------------------
> Fold 5 - Loss: 0.8477826118469238 - Accuracy: 80.59701323509216%
> Fold 5 - Precision: 0.9069767441860465 - Recall: 0

## 3 CLASS CLASSIFICATION USING LSTM

In [17]:
#control_data, control_labels = get_all_td_features(control_dir, 0)
#als_with_data, als_with_labels = get_all_td_features(als_with_dir, 1)
#als_without_data, als_without_labels = get_all_td_features(als_without_dir, 2)

control_data, control_labels = get_MFCC_data(control_dir, 0)
als_with_data, als_with_labels = get_MFCC_data(als_with_dir, 1)
als_without_data, als_without_labels = get_MFCC_data(als_without_dir, 2)


X = np.concatenate([control_data, als_with_data, als_without_data], axis=0)
y = np.concatenate([control_labels, als_with_labels, als_without_labels], axis=0)
X = np.array(X)


# Data Cleaning
"""
X[X == None] = np.nan
X = np.nan_to_num(X, nan=0.0)
df = pd.DataFrame(X)
df.fillna(0.0, inplace=True)
X = df.to_numpy()
X = X.reshape(-1,10,1)
"""

X = X.reshape(-1,128,1)
X, y = shuffle(X, y, random_state=0)

100%|██████████| 176/176 [00:04<00:00, 38.57it/s]


In [18]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(128, 1)))  # 64 units in LSTM layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation='softmax'))  # Output layer with 1 neuron for binary classification

# Print model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                

In [19]:
num_folds = 10
fold_no = 1
kfold = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []
loss_per_fold = []

# Model Hyperparameters Turning
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1,
                                                 restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('lstm_three_class_model.h5', monitor='loss', verbose=1,
                                                      save_best_only=True)

for train, test in kfold.split(X, y):
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('---------------------------------------------------')

    history = model.fit(X[train], y[train], epochs=100, verbose=1,
                        callbacks=[earlystopping, model_checkpoint])

    scores = model.evaluate(X[test], y[test], verbose=0)


    y_pred = model.predict(X[test])
    y_pred  = np.argmax(y_pred, axis=-1)
    precision = precision_score(y[test], y_pred, average='weighted')
    recall = recall_score(y[test], y_pred, average='weighted')
    f1 = f1_score(y[test], y_pred, average='weighted')
    
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold.append(scores[1] * 100)
    precision_per_fold.append(precision)
    recall_per_fold.append(recall)
    f1_per_fold.append(f1)
    loss_per_fold.append(scores[0])
    """"
    if fold_no == 9:

        class_names = ['Control', 'ALS_with_dysarthria', 'ALS_without_dysarthria']
        confusion = confusion_matrix(y[test], y_pred)
        plot_confusion_matrix(conf_mat=confusion, figsize=(12,8), class_names=class_names, show_normed=True, hide_ticks=True, cmap=plt.cm.Blues)
    """
    fold_no += 1

---------------------------------------------------
Epoch 1/100
19/19 [==============================] - ETA: 0s - loss: 1.0928 - accuracy: 0.4324
Epoch 1: loss improved from inf to 1.09278, saving model to lstm_three_class_model.h5
19/19 [==============================] - 2s 34ms/step - loss: 1.0928 - accuracy: 0.4324
Epoch 2/100
19/19 [==============================] - ETA: 0s - loss: 1.0805 - accuracy: 0.4407
Epoch 2: loss improved from 1.09278 to 1.08052, saving model to lstm_three_class_model.h5
19/19 [==============================] - 1s 34ms/step - loss: 1.0805 - accuracy: 0.4407
Epoch 3/100
19/19 [==============================] - ETA: 0s - loss: 1.0723 - accuracy: 0.4407
Epoch 3: loss improved from 1.08052 to 1.07232, saving model to lstm_three_class_model.h5
19/19 [==============================] - 1s 35ms/step - loss: 1.0723 - accuracy: 0.4407
Epoch 4/100
19/19 [==============================] - ETA: 0s - loss: 1.0673 - accuracy: 0.4424
Epoch 4: loss improved from 1.07232 to

In [20]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
    print(f'> Fold {i+1} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')

Average Score per fold 
-----------------------------------------------
> Fold 1 - Loss: 0.9398107528686523 - Accuracy: 59.70149040222168%
> Fold 1 - Precision: 0.6022638006141541 - Recall: 0.5970149253731343 - F1: 0.5994262919636053
-----------------------------------------------
> Fold 2 - Loss: 0.7249385118484497 - Accuracy: 76.11940503120422%
> Fold 2 - Precision: 0.7951492537313433 - Recall: 0.7611940298507462 - F1: 0.7652061436460665
-----------------------------------------------
> Fold 3 - Loss: 0.45280370116233826 - Accuracy: 80.59701323509216%
> Fold 3 - Precision: 0.8577699736611062 - Recall: 0.8059701492537313 - F1: 0.8161645132546225
-----------------------------------------------
> Fold 4 - Loss: 0.49762600660324097 - Accuracy: 85.07462739944458%
> Fold 4 - Precision: 0.8583848434594703 - Recall: 0.8507462686567164 - F1: 0.8513950098824914
-----------------------------------------------
> Fold 5 - Loss: 0.46507421135902405 - Accuracy: 82.08954930305481%
> Fold 5 - Precisi